## 3.1 Preparation

### Import

In [1]:
import pandas as pd

import pymysql
import json
import pymongo

import time
import json

### Connecting to MongoDB

In [53]:
DATABASE_NAME = "DBMS"
COLLECTION_NAME = "tweet"

client = pymongo.MongoClient()
db = client[DATABASE_NAME]
tweets = db[COLLECTION_NAME]

#### test for tweets load

In [3]:
documents = tweets.find().limit(5)
pd.DataFrame(documents)

,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,644aeb6165c7444702bc28fc,"{'id_str': '804046791348015107', 'created_at':...","É isto, ou vou morrer sem ar ou com o corona h...","{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022770679320576,None,None,None,None,0,0,0,0,pt
1,644aeb6165c7444702bc28fd,"{'id_str': '2242948745', 'created_at': '2013-1...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",2020-04-25,1254022770746372096,None,None,None,None,0,0,0,0,de
2,644aeb6165c7444702bc28fe,"{'id_str': '2929344220', 'created_at': '2014-1...",Was sollen 150 Euro Computerzuschuss bringen? ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022772575043586,None,None,None,1254016434658848769,0,0,0,0,de
3,644aeb6165c7444702bc28ff,"{'id_str': '1091660129894838272', 'created_at'...",@VinceMcMahon @TripleH We hereby honor to anno...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022773598572544,VinceMcMahon,1222639789,1253768179517657088,None,0,0,0,0,en
4,644aeb6165c7444702bc2900,"{'id_str': '375777294', 'created_at': '2011-09...",im making 17 a hr doing nothing 😭😭😭😭 i love th...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022776094105602,None,None,None,None,0,0,0,0,en


### Connecting to MySQL

In [3]:
#Connect to MySQL
conn = pymysql.connect(host='localhost', port=3306, user='root',password="19989797",charset="utf8mb4", database='db1')
cur = conn.cursor()

In [4]:
cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users;")
users = pd.DataFrame(cur.fetchall())
users.columns = [desc[0] for desc in cur.description]
users.head(10)

,userid_str,screen_name,name,favourites_count
0,1000006582896295938,yourgirlsarag,sara,11349
1,1000027886915637250,cheche04125843,cheche,1031
2,1000034375973646337,clarice8967,clarih ?,40141
3,100004211,karundhiravidan,azakiya tamilmagan,70330
4,1000045322859634688,barcelona9581,barcelona958,29449
5,1000046756120363013,isisevang,áisis,9384
6,1000047984103575552,esesesrarara34,EsRaAa (EN YENI HESAP)?,1459
7,1000055605263065088,zolesa_ke,Tsika Zolesa,37216
8,1000061119921352705,lejayk54,☮️Axel☮️,1438
9,1000062434126725120,mahin37418807,Mahin Mohades,5232


## 3.2 Single Queries in NRDB for the search application

### 3.2.1 Search by keyword

In [17]:
#Find tweets with keyword
tweets.create_index([("text", pymongo.TEXT)])
def search_word():
    user_input = input("Please enter a keyword: ")
    # search in NRDB
    if user_input != "":
        myquery = {"$text":{"$search": user_input}}
        twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(users, twts, on='userid_str')
    return testItems

In [21]:
search_word()

Please enter a keyword: RT


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,1135583492702687234,wasthend,William Almeida,935,644aeba865c7444702bcba5e,"{'id_str': '1135583492702687234', 'created_at'...",Por favor deem Rt e denunciem.,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254056652610945025,None,None,None,1254038813195800576,0,0,0,0,pt
1,1149671626616610817,PrabhakarKolla2,Prabhakar Kollapudi,945,644aeb9065c7444702bc8a96,"{'id_str': '1149671626616610817', 'created_at'...",Without tabilighi india would have been corona...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254045968443437056,None,None,None,None,0,0,0,0,en
2,1221778286700679170,naolanic,BELI ROTI KERING NAO DONG CEK PINNED/GI LIMIT SBR,2952,644aeb7065c7444702bc4726,"{'id_str': '1221778286700679170', 'created_at'...",@stmaemunah05 @radenrauf https://t.co/5WZm9h1c...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254030251157413888,stmaemunah05,2215623222,1254030070475141121,1250610299004055554,0,0,0,0,in
3,1221778286700679170,naolanic,BELI ROTI KERING NAO DONG CEK PINNED/GI LIMIT SBR,2952,644aeb8c65c7444702bc8181,"{'id_str': '1221778286700679170', 'created_at'...",@GRESAIDS https://t.co/gOJ8ltsonR bantu RT Kak...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254043945933262851,GRESAIDS,1101162508583223297,1254018889366683648,1250610299004055554,0,0,0,0,in
4,1366789854,astapidrullah,.pid,11273,644aeb7865c7444702bc57fa,"{'id_str': '1366789854', 'created_at': '2013-0...",Bacot lo mas mas sirkel rt.,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254034190070214656,None,None,None,1253939460078592000,0,0,0,0,in
5,2306048802,OgiMontena_,HeyJude,1477,644aeb6465c7444702bc2da9,"{'id_str': '2306048802', 'created_at': '2014-0...",Oh sirkel rt ckck pantes kek kontol,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254023975379677185,None,None,None,1253939460078592000,0,0,0,0,in
6,2914383445,manameismarshal,???????,358,644aeb8e65c7444702bc863a,"{'id_str': '2914383445', 'created_at': '2014-1...",Anjay ada yang positif corona di rt aku,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254045023278002176,None,None,None,None,0,0,0,0,in
7,3950306362,ViihMss15,Vi Gabs,882,644aebaa65c7444702bcc0a6,"{'id_str': '3950306362', 'created_at': '2015-1...",Dêem rt e denunciem,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254058043282784256,None,None,None,1254038813195800576,0,0,0,0,pt
8,632834693,ivey00880866,ivey,23607,644aeba265c7444702bcae29,"{'id_str': '632834693', 'created_at': '2012-07...",RT,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254053961780387840,None,None,None,1251938514670948354,0,0,0,0,und
9,979285512040009728,Thaladhoniabar1,Thaladhoniveriyan?,4509,644aeb7665c7444702bc531a,"{'id_str': '979285512040009728', 'created_at':...",#அன்புள்ளசூர்யா Rt potavanuku... Corona varath...,"{'hashtags': [{'text': 'அன்புள்ளசூர்யா', 'indi...",2020-04-25,1254033090571165698,None,None,None,None,0,0,0,0,und


### 3.2.2 Search by hashtag

In [34]:
#Find tweets with hastage
def search_hashtag():
    user_input = input("Please enter a hashtag: ")
    if user_input !="":
        myquery = {"entities.hashtags.text": user_input}
        twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(users, twts, on='userid_str')
    return testItems

In [35]:
search_hashtag()

Please enter a hashtag: sport


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,2242948745,tho1965,Thomas Krause,2184,644aeb6165c7444702bc28fd,"{'id_str': '2242948745', 'created_at': '2013-1...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",2020-04-25,1254022770746372096,None,None,None,None,0,0,0,0,de


### 3.2.3 Search by user name

In [36]:
def search_user_name():
    user_input = input("Please enter a user_name: ")
    if user_input !="":
        # my sql
        myquery = "SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;"
        cur.execute(myquery,(user_input,))
        uers = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
        # mongodb
        userid_str=uers["userid_str"].tolist()
        myquery = {"user.id_str": {"$in":userid_str}}
        twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
        #Processing
        twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
        #JOIN
        testItems = pd.merge(users, twts, on='userid_str')
        return testItems

In [37]:
search_user_name()

Please enter a user_name: tho1965


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,2242948745,tho1965,Thomas Krause,2184,644aeb6165c7444702bc28fd,"{'id_str': '2242948745', 'created_at': '2013-1...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",2020-04-25,1254022770746372096,None,None,None,None,0,0,0,0,de


### 3.2.4 Search by time range

In [38]:
def search_date():
    myquery={}
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    return twts

In [39]:
search_date()

Please enter a start date(format:yyyy-mm-dd hh:mm:ss): 2020-04-20
Please enter a end date(format:yyyy-mm-dd hh:mm:ss): 2020-04-30


,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,644aeb6165c7444702bc2904,"{'id_str': '1035107587006648320', 'created_at'...",पश्चिम बंगाल सरकार ने पहली बार माना है कि हाल ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022779608891393,None,None,None,None,0,0,0,0,hi
1,644aeb6165c7444702bc2905,"{'id_str': '923169415054680064', 'created_at':...",Weekly mortality graphs show in some European ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022780695252993,None,None,None,None,0,0,0,0,en
2,644aeb6165c7444702bc2900,"{'id_str': '375777294', 'created_at': '2011-09...",im making 17 a hr doing nothing 😭😭😭😭 i love th...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022776094105602,None,None,None,None,0,0,0,0,en
3,644aeb6165c7444702bc2903,"{'id_str': '301470336', 'created_at': '2011-05...","ahap , low cut off merchants","{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022778371571712,None,None,None,1254016297161105408,0,0,0,0,en
4,644aeb6165c7444702bc2901,"{'id_str': '865733987561381888', 'created_at':...",@MichaelTouby @ChinaDaily @RVsmtown @YRFairyta...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022776207429633,MichaelTouby,719625570615758849,1254013543470563328,None,0,0,0,0,en
5,644aeb6165c7444702bc28ff,"{'id_str': '1091660129894838272', 'created_at'...",@VinceMcMahon @TripleH We hereby honor to anno...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022773598572544,VinceMcMahon,1222639789,1253768179517657088,None,0,0,0,0,en
6,644aeb6165c7444702bc28fd,"{'id_str': '2242948745', 'created_at': '2013-1...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",2020-04-25,1254022770746372096,None,None,None,None,0,0,0,0,de
7,644aeb6165c7444702bc28fc,"{'id_str': '804046791348015107', 'created_at':...","É isto, ou vou morrer sem ar ou com o corona h...","{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022770679320576,None,None,None,None,0,0,0,0,pt
8,644aeb6165c7444702bc28fe,"{'id_str': '2929344220', 'created_at': '2014-1...",Was sollen 150 Euro Computerzuschuss bringen? ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022772575043586,None,None,None,1254016434658848769,0,0,0,0,de
9,644aeb6165c7444702bc2902,"{'id_str': '1132273796138905600', 'created_at'...",Oh brother and he gave the kid his corona type...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022776752615430,None,None,None,None,0,0,0,0,en


## 3.3 Search Function by all types

#### 3.3.1 using no cache

In [13]:
def search_all_RNR():
    myquery={}
    search_keyword = input("Please enter a keyword: ")
    search_hashtag = input("Please enter a hashtag: ")
    search_user = input("Please enter a user_name: ")
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    # screen name
    if search_user != "":
        # my sql
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (search_user,))
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
        # mongodb
        userid_str = uer["userid_str"].tolist()
        myquery["user.id_str"] = {"$in":userid_str}
    else:
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users;")
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
    if search_keyword != "":
        myquery["$text"] = {"$search": search_keyword}
    if search_hashtag != "":
        myquery["entities.hashtags.text"] = search_hashtag
    # time range
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(uer, twts, on='userid_str')
    return testItems

###### take a view

In [51]:
search_all_RNR()

Please enter a keyword: 
Please enter a hashtag: 
Please enter a user_name: 
Please enter a start date(format:yyyy-mm-dd hh:mm:ss): 
Please enter a end date(format:yyyy-mm-dd hh:mm:ss): 


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,1035107587006648320,SatyaHindi,Satya Hindi,51,644c5d9ad23998d6c8f335d6,"{'id': 1.0351075870066483e+18, 'id_str': '1035...",पश्चिम बंगाल सरकार ने पहली बार माना है कि हाल ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",Sat Apr 25 12:21:43 +0000 2020,1254022779608891393,None,None,None,None,0,0,0,0,hi
1,1091660129894838272,milliteknoloj,Milli teknoloji,352,644c5d9ad23998d6c8f335d1,"{'id': 1.0916601298948383e+18, 'id_str': '1091...",@VinceMcMahon @TripleH We hereby honor to anno...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",Sat Apr 25 12:21:42 +0000 2020,1254022773598572544,VinceMcMahon,1222639789,1253768179517657088,None,0,0,0,0,en
2,1132273796138905600,RampTheresa,Terri Kamp,29166,644c5d9ad23998d6c8f335d4,"{'id': 1.1322737961389056e+18, 'id_str': '1132...",Oh brother and he gave the kid his corona type...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",Sat Apr 25 12:21:43 +0000 2020,1254022776752615430,None,None,None,None,0,0,0,0,en
3,2242948745,tho1965,Thomas Krause,2184,644c5d9ad23998d6c8f335cf,"{'id': 2242948745.0, 'id_str': '2242948745', '...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",Sat Apr 25 12:21:41 +0000 2020,1254022770746372096,None,None,None,None,0,0,0,0,de
4,2929344220,RusticusArat,Ralf Schmitz,32024,644c5d9ad23998d6c8f335d0,"{'id': 2929344220.0, 'id_str': '2929344220', '...",Was sollen 150 Euro Computerzuschuss bringen? ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",Sat Apr 25 12:21:42 +0000 2020,1254022772575043586,None,None,None,1254016434658848769,0,0,0,0,de
5,301470336,_FreshAA,BABA ALI,857,644c5d9ad23998d6c8f335d5,"{'id': 301470336, 'id_str': '301470336', 'name...","ahap , low cut off merchants","{'hashtags': [], 'urls': [], 'user_mentions': ...",Sat Apr 25 12:21:43 +0000 2020,1254022778371571712,None,None,None,1254016297161105408,0,0,0,0,en
6,375777294,TWD40_,TeéLaneeë?,1325,644c5d9ad23998d6c8f335d2,"{'id': 375777294, 'id_str': '375777294', 'name...",im making 17 a hr doing nothing 😭😭😭😭 i love th...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",Sat Apr 25 12:21:43 +0000 2020,1254022776094105602,None,None,None,None,0,0,0,0,en
7,804046791348015107,B_King69,Bi Sex Uau,5446,644c5d9ad23998d6c8f335ce,"{'id': 8.040467913480151e+17, 'id_str': '80404...","É isto, ou vou morrer sem ar ou com o corona h...","{'hashtags': [], 'urls': [], 'user_mentions': ...",Sat Apr 25 12:21:41 +0000 2020,1254022770679320576,None,None,None,None,0,0,0,0,pt
8,865733987561381888,BritishKatieKim,ANH,244132,644c5d9ad23998d6c8f335d3,"{'id': 8.657339875613819e+17, 'id_str': '86573...",@MichaelTouby @ChinaDaily @RVsmtown @YRFairyta...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",Sat Apr 25 12:21:43 +0000 2020,1254022776207429633,MichaelTouby,719625570615758849,1254013543470563328,None,0,0,0,0,en
9,923169415054680064,QuantumAspect,Quantum,119803,644c5d9ad23998d6c8f335d7,"{'id': 9.2316941505468e+17, 'id_str': '9231694...",Weekly mortality graphs show in some European ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",Sat Apr 25 12:21:44 +0000 2020,1254022780695252993,None,None,None,None,0,0,0,0,en


#### 3.3.2 using cache

In [14]:
def search_all_C():
    myquery={}
    search_keyword = input("Please enter a keyword: ")
    search_hashtag = input("Please enter a hashtag: ")
    search_user = input("Please enter a user_name: ")
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    # screen name
    if search_user != "":
        # my sql
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (search_user,))
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
        # mongodb
        userid_str = uer["userid_str"].tolist()
        myquery["user.id_str"] = {"$in":userid_str}
    else:
        uer = users
    if search_keyword != "":
        myquery["$text"] = {"$search": search_keyword}
    if search_hashtag != "":
        myquery["entities.hashtags.text"] = search_hashtag
    # time range
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(uer, twts, on='userid_str')
    return testItems

###### take a view

In [15]:
search_all_C()

Please enter a keyword: 
Please enter a hashtag: 
Please enter a user_name: tho1965
Please enter a start date(format:yyyy-mm-dd hh:mm:ss): 
Please enter a end date(format:yyyy-mm-dd hh:mm:ss): 


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,2242948745,tho1965,Thomas Krause,2184,644aeb6165c7444702bc28fd,"{'id_str': '2242948745', 'created_at': '2013-1...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",2020-04-25,1254022770746372096,None,None,None,None,0,0,0,0,de


## 3.4 time test

#### for uncached data

In [30]:
def test_uncached_performance():
    myquery={}
    search_keyword = input("Please enter a keyword: ")
    search_hashtag = input("Please enter a hashtag: ")
    search_user = input("Please enter a user_name: ")
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    # search start
    start_time = time.time()
    # screen name
    if search_user != "":
        # my sql
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (search_user,))
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
        # mongodb
        userid_str = uer["userid_str"].tolist()
        myquery["user.id_str"] = {"$in":userid_str}
    else:
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users;")
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
    if search_keyword != "":
        myquery["$text"] = {"$search": search_keyword}
    if search_hashtag != "":
        myquery["entities.hashtags.text"] = search_hashtag
    # time range
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(uer, twts, on='userid_str')
    # search end
    end_time = time.time()
    print(f''' searching in cached information took {end_time - start_time:.4f} seconds''')
    return testItems

In [55]:
test_uncached_performance()

Please enter a keyword: RT
Please enter a hashtag: 
Please enter a user_name: 
Please enter a start date(format:yyyy-mm-dd hh:mm:ss): 
Please enter a end date(format:yyyy-mm-dd hh:mm:ss): 
 searching in cached information took 0.6023 seconds


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,1135583492702687234,wasthend,William Almeida,935,644aeba865c7444702bcba5e,"{'id_str': '1135583492702687234', 'created_at'...",Por favor deem Rt e denunciem.,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254056652610945025,None,None,None,1254038813195800576,0,0,0,0,pt
1,1149671626616610817,PrabhakarKolla2,Prabhakar Kollapudi,945,644aeb9065c7444702bc8a96,"{'id_str': '1149671626616610817', 'created_at'...",Without tabilighi india would have been corona...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254045968443437056,None,None,None,None,0,0,0,0,en
2,1221778286700679170,naolanic,BELI ROTI KERING NAO DONG CEK PINNED/GI LIMIT SBR,2952,644aeb7065c7444702bc4726,"{'id_str': '1221778286700679170', 'created_at'...",@stmaemunah05 @radenrauf https://t.co/5WZm9h1c...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254030251157413888,stmaemunah05,2215623222,1254030070475141121,1250610299004055554,0,0,0,0,in
3,1221778286700679170,naolanic,BELI ROTI KERING NAO DONG CEK PINNED/GI LIMIT SBR,2952,644aeb8c65c7444702bc8181,"{'id_str': '1221778286700679170', 'created_at'...",@GRESAIDS https://t.co/gOJ8ltsonR bantu RT Kak...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254043945933262851,GRESAIDS,1101162508583223297,1254018889366683648,1250610299004055554,0,0,0,0,in
4,1366789854,astapidrullah,.pid,11273,644aeb7865c7444702bc57fa,"{'id_str': '1366789854', 'created_at': '2013-0...",Bacot lo mas mas sirkel rt.,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254034190070214656,None,None,None,1253939460078592000,0,0,0,0,in
5,2306048802,OgiMontena_,HeyJude,1477,644aeb6465c7444702bc2da9,"{'id_str': '2306048802', 'created_at': '2014-0...",Oh sirkel rt ckck pantes kek kontol,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254023975379677185,None,None,None,1253939460078592000,0,0,0,0,in
6,2914383445,manameismarshal,???????,358,644aeb8e65c7444702bc863a,"{'id_str': '2914383445', 'created_at': '2014-1...",Anjay ada yang positif corona di rt aku,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254045023278002176,None,None,None,None,0,0,0,0,in
7,3950306362,ViihMss15,Vi Gabs,882,644aebaa65c7444702bcc0a6,"{'id_str': '3950306362', 'created_at': '2015-1...",Dêem rt e denunciem,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254058043282784256,None,None,None,1254038813195800576,0,0,0,0,pt
8,632834693,ivey00880866,ivey,23607,644aeba265c7444702bcae29,"{'id_str': '632834693', 'created_at': '2012-07...",RT,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254053961780387840,None,None,None,1251938514670948354,0,0,0,0,und
9,979285512040009728,Thaladhoniabar1,Thaladhoniveriyan?,4509,644aeb7665c7444702bc531a,"{'id_str': '979285512040009728', 'created_at':...",#அன்புள்ளசூர்யா Rt potavanuku... Corona varath...,"{'hashtags': [{'text': 'அன்புள்ளசூர்யா', 'indi...",2020-04-25,1254033090571165698,None,None,None,None,0,0,0,0,und


#### for cached data

In [32]:
def test_cached_performance():
    myquery={}
    search_keyword = input("Please enter a keyword: ")
    search_hashtag = input("Please enter a hashtag: ")
    search_user = input("Please enter a user_name: ")
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    # search start
    start_time = time.time()
    # search in NRDB
    # screen name
    if search_user != "":
        # my sql
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (search_user,))
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
        # mongodb
        userid_str = uer["userid_str"].tolist()
        myquery["user.id_str"] = {"$in":userid_str}
    else:
        uer = users
    if search_keyword != "":
        myquery["$text"] = {"$search": search_keyword}
    if search_hashtag != "":
        myquery["entities.hashtags.text"] = search_hashtag
    # time range
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(uer, twts, on='userid_str')
    # search end
    end_time = time.time()      
    print(f''' searching in cached information took {end_time - start_time:.4f} seconds''')
    return testItems

In [56]:
test_cached_performance()

Please enter a keyword: RT
Please enter a hashtag: 
Please enter a user_name: 
Please enter a start date(format:yyyy-mm-dd hh:mm:ss): 
Please enter a end date(format:yyyy-mm-dd hh:mm:ss): 
 searching in cached information took 0.0279 seconds


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,1135583492702687234,wasthend,William Almeida,935,644aeba865c7444702bcba5e,"{'id_str': '1135583492702687234', 'created_at'...",Por favor deem Rt e denunciem.,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254056652610945025,None,None,None,1254038813195800576,0,0,0,0,pt
1,1149671626616610817,PrabhakarKolla2,Prabhakar Kollapudi,945,644aeb9065c7444702bc8a96,"{'id_str': '1149671626616610817', 'created_at'...",Without tabilighi india would have been corona...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254045968443437056,None,None,None,None,0,0,0,0,en
2,1221778286700679170,naolanic,BELI ROTI KERING NAO DONG CEK PINNED/GI LIMIT SBR,2952,644aeb7065c7444702bc4726,"{'id_str': '1221778286700679170', 'created_at'...",@stmaemunah05 @radenrauf https://t.co/5WZm9h1c...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254030251157413888,stmaemunah05,2215623222,1254030070475141121,1250610299004055554,0,0,0,0,in
3,1221778286700679170,naolanic,BELI ROTI KERING NAO DONG CEK PINNED/GI LIMIT SBR,2952,644aeb8c65c7444702bc8181,"{'id_str': '1221778286700679170', 'created_at'...",@GRESAIDS https://t.co/gOJ8ltsonR bantu RT Kak...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254043945933262851,GRESAIDS,1101162508583223297,1254018889366683648,1250610299004055554,0,0,0,0,in
4,1366789854,astapidrullah,.pid,11273,644aeb7865c7444702bc57fa,"{'id_str': '1366789854', 'created_at': '2013-0...",Bacot lo mas mas sirkel rt.,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254034190070214656,None,None,None,1253939460078592000,0,0,0,0,in
5,2306048802,OgiMontena_,HeyJude,1477,644aeb6465c7444702bc2da9,"{'id_str': '2306048802', 'created_at': '2014-0...",Oh sirkel rt ckck pantes kek kontol,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254023975379677185,None,None,None,1253939460078592000,0,0,0,0,in
6,2914383445,manameismarshal,???????,358,644aeb8e65c7444702bc863a,"{'id_str': '2914383445', 'created_at': '2014-1...",Anjay ada yang positif corona di rt aku,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254045023278002176,None,None,None,None,0,0,0,0,in
7,3950306362,ViihMss15,Vi Gabs,882,644aebaa65c7444702bcc0a6,"{'id_str': '3950306362', 'created_at': '2015-1...",Dêem rt e denunciem,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254058043282784256,None,None,None,1254038813195800576,0,0,0,0,pt
8,632834693,ivey00880866,ivey,23607,644aeba265c7444702bcae29,"{'id_str': '632834693', 'created_at': '2012-07...",RT,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254053961780387840,None,None,None,1251938514670948354,0,0,0,0,und
9,979285512040009728,Thaladhoniabar1,Thaladhoniveriyan?,4509,644aeb7665c7444702bc531a,"{'id_str': '979285512040009728', 'created_at':...",#அன்புள்ளசூர்யா Rt potavanuku... Corona varath...,"{'hashtags': [{'text': 'அன்புள்ளசூர்யா', 'indi...",2020-04-25,1254033090571165698,None,None,None,None,0,0,0,0,und
